In [ ]:
import pandas as pd
import numpy as np
import igraph as ig

# Import the city data.
raw = []
with open('input.txt','r') as f:
    for line in f:
        if line.strip()[0] != "#":
            raw.append(line.split())
            
n_cities = int(raw[0][0])
reliability_1d = np.asarray(raw[1], dtype='double')
cost_1d = np.asarray(raw[2], dtype='int')

# Convert the 1D matrices to 2D.
def convert_to_2d(size, arr1d):
    new_array = np.zeros((size, size))
    k = 0
    for i in range(0, size):
        for j in range(i + 1, size):
            new_array[i][j] = arr1d[k]
            k = k + 1
    return new_array

reliability = convert_to_2d(n_cities, reliability_1d)
cost = convert_to_2d(n_cities, cost_1d)

# Print matrices and extracted information.
print("Number of cities: " + str(n_cities))
print("Reliability matrix:")
print(reliability)
print("Cost matrix:")
print(cost)

# Create reliability graph.
reliability_graph = ig.Graph.Weighted_Adjacency(reliability).as_undirected()
reliability_graph.es["label"] = [str(i) for i in reliability_1d]
reliability_graph.es["curved"] = [False for i in reliability_1d]
reliability_graph.vs["label"] = range(n_cities)

# Create cost graph.
cost_graph = ig.Graph.Weighted_Adjacency(cost).as_undirected()
cost_graph.es["label"] = [str(i) for i in cost_1d]
cost_graph.es["curved"] = [False for i in cost_1d]
cost_graph.vs["label"] = range(n_cities)

In [ ]:
# Plot reliability graph.
ig.plot(reliability_graph)

In [ ]:
# Plot cost graph.
ig.plot(cost_graph)